In [1]:
#Usual Imports
import os
import sys
import time
import datetime
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Imports for web scraping Spotify data
from bs4 import BeautifulSoup
import urllib
import requests
from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from time import sleep
from collections import Counter

In [2]:
# Initialize Spotipy API credentials
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [31]:
# Read in billboard top 100 data from 1955-2015
billboard_data=pd.read_csv("all_billboard_data.txt", sep="|", encoding='latin1')
#Arrange Billboard data by tracks for Spotify web scraping 
billboard_tracks = billboard_data.groupby(["title","artist"]).size().reset_index()

In [4]:
# We have 34605 unique songs in our dataset
len(billboard_tracks)

34605

In [8]:
# Initialize an empty data frame for the Spotify data
track_meta_columns=["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","type","id","uri","track_href","analysis_url","duration_ms","time_signature"]
artist_data = pd.DataFrame(columns=["artist","track"])
track_data = pd.DataFrame(columns=track_meta_columns)
spotify_data=pd.concat([artist_data, track_data], axis=1)
artist_index=0
track_index=0

In [6]:
# Obtaining Data Step
spotify_data=pd.concat([artist_data, track_data], axis=1)
lower_limit =0
upper_limit =150
spot_table = 'spotify' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
# Cycle through the rows in the billboard_tracks dataframe.
# Search for the artist and title for each one through Spotify's API
# Add the metadata for the track to the dataset
for row in range(lower_limit,upper_limit):
    artist_data = pd.DataFrame(columns=["artist","track"])
    track_data = pd.DataFrame(columns=track_meta_columns)
    artist_name = billboard_tracks.iloc[row,1]
    track_name = billboard_tracks.iloc[row,0]
    lookup = artist_name + " " + track_name
    try:
        results = sp.search(q=lookup, limit=3)
    except urllib.request.HTTPError as eone:
        pass
    tids = []
    artist_index=0
    for i, t in enumerate(results['tracks']['items']):
        to_add =pd.DataFrame( {"artist":t['artists'][0]['name'], "track":t['name']},index=[artist_index])
        artist_data=pd.concat([artist_data,to_add])
        tids.append(t['uri'])
        artist_index+=1
    try:    
        features = sp.audio_features(tids)
    except urllib.request.HTTPError as etwo:
        pass    
    track_index=0
    for feature in features:
        tracks = pd.DataFrame(feature, index=[track_index])
        track_data=pd.concat([track_data,tracks])
        track_index+=1
    all_data = pd.concat([artist_data, track_data], axis=1)
    spotify_data=pd.concat([spotify_data,all_data])
# Spotify has rate limiting for their API so we must delay time between requests
    if row%50==0 and row!=lower_limit and row!=upper_limit:
        sleep(25)
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
# Output the data to a csv file
# Note that due to the length of time required we will create several csv files
spotify_data.to_csv(spot_table)

retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
features retrieved in 113.41 seconds


In [ ]:
# Cleaning and Wrangling step
# Begin to join all the csv files together, some may require adding in an additional empty row.
spot_one = pd.read_csv('spotify0-50.csv',  encoding='latin1')
spot_two = pd.read_csv('spotify50-100.csv',  encoding='latin1')
spotify_all = pd.concat([spot_one,spot_two])
spot_new = pd.read_csv('spotify100-200.csv',  encoding='latin1')
spotify_all = pd.concat([spotify_all,spot_new])
spotify_empty = spot_small[87:88]
spotify_all = pd.concat([spotify_all,spotify_empty])
# spotify_all.to_csv("spotify_big")

In [28]:
# Read in Large Spotify dataset
big_spotify = pd.read_csv('spotify_big.csv',  encoding='latin1')
# We will match the billboard_track data and the spotify data one-to-one
# We only require one result per track
spotify_merge = big_spotify[big_spotify["Unnamed: 0.1"]==0]
# Subset interesting columns
spotify_merge =spotify_merge[["acousticness","artist","danceability","duration_ms","energy","key","loudness","mode","track","instrumentalness","liveness","valence","tempo","time_signature"]]
# Rename artist column
spotify_merge=spotify_merge.rename(columns = {'artist':'spotify_artist'})
spotify_merge=spotify_merge.reset_index()
len(spotify_merge)

34607

In [33]:
# Drop two extra rows
spotify_merge=spotify_merge.drop(spotify_merge.index[[101,201]]).reset_index()
# Merge data with the billboard_data
merged_data=pd.concat([billboard_tracks, spotify_merge], axis=1)
# Join data to billboard_list
all_data=pd.merge(billboard_data,merged_data,how='left', on=['artist', 'title'])

In [37]:
spotify_merge.to_csv("spotify_scraped.csv")

In [43]:
# Initial succes rate
len(spotify_merge[spotify_merge.spotify_artist.notnull()])/len(spotify_merge)

0.7400086692674469

In [14]:
#Read in datasets from Decibels & Decimals
billboard_data = pd.read_csv("all_charts.csv")
spotify_data = pd.read_csv("audio_features.csv")

C:\Users\MSI_USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
billboard_data.info()
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304643 entries, 0 to 304642
Data columns (total 30 columns):
Unnamed: 0     304643 non-null int64
chartDate      304643 non-null object
title          304643 non-null object
artist         304643 non-null object
peakPos        304643 non-null int64
lastPos        304643 non-null int64
weeks          304643 non-null int64
rank           304643 non-null int64
change         304643 non-null object
spotifyID      258820 non-null object
spotifyLink    258820 non-null object
videoLink      109357 non-null object
artist_list    258820 non-null object
track_name     258668 non-null object
art_0          258668 non-null object
art_1          27213 non-null object
art_2          5337 non-null object
art_3          1472 non-null object
art_4          353 non-null object
art_5          93 non-null object
art_6          34 non-null object
art_7          14 non-null object
art_8          11 non-null object
art_9          11 non-null object
art_10    

In [20]:
# For this project we only need the chartDate, title, artist, peakPos, lastPos, weeks, rank, change and 
# spotifyLink columns from billboard_data and the acousticness, danceability, duration_ms, energy, id,
# instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature and valence columns
# from the spotify_data
billboard_sub = billboard_data[["chartDate","title","artist","peakPos","lastPos","weeks","rank","change","spotifyID"]]
spotify_sub = spotify_data[["acousticness", "danceability", "duration_ms", "energy", "id","instrumentalness", "key", "liveness", "loudness", "mode", "speechiness", "tempo", "time_signature","valence"]]

In [25]:
# We merged the datasets from Decibels & Decimals together and create a new csv file
merged_data=pd.merge(billboard_sub,spotify_sub, how="outer",left_on="spotifyID",right_on="id")
merged_data.to_csv("all_spotify_data.csv")

In [2]:
# Read in the merged datasets from Decibels & Decimals and create a subset of individual tracks 
new_big_data=pd.read_csv("all_spotify_data.csv",encoding='latin1')

In [46]:
search_data=new_big_data.groupby(["title","artist"]).size().reset_index()
len(search_data)

27588

In [40]:
# We have Spotify Data for 78.33% of the tracks in the Billboard Top 100.
len(spotify_data)/len(search_data)*100

78.33478323908946

In [34]:
lyrics_data = pd.DataFrame()
lower_limit =0
upper_limit =27588
lyrics_table = 'lyrics-' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
# We use the same procedure as before to obtain the lyrics data for the songs.
# We cycle through the rows in the search_data dataframe.
# We search for the lyric data for each track and add the lyrics to the lyrics_data dataframe.
for row in range(lower_limit,upper_limit):
    artist_raw = search_data.iloc[row,1]
    artist_noand=artist_raw.replace(" &","").replace(". ","-").replace("Ã©","e").replace("$", "s")
    artist_array = re.split(r'.Featuring+', artist_noand) # Consider only the first listed artist
    track_raw=search_data.iloc[row,0]
    # Replace extended latin characters
    track_name=track_raw.replace("Ã\x83Â¶","oe").replace("Ã©","e").replace("ÃÂ©","e").replace("ÃÂ³",'o')
    track_name=track_name.replace("ÃÂ¡","a").replace("ÃÂ±","n").replace("Ã\x83â\x80\x9c","o").replace("ÃÂ¿DÃÂ²","do").replace("ÃÂ","a")
    track_name=re.sub(r'\&\s', '', track_name).replace("$", "s").replace("#","") # Replace certain characters
    track_name=re.sub(r'S\*\*t', 'Shit',track_name) # Replace profanity
    track_name=re.sub(r'\(|\)|\'|\|\.|\"|\,', '',track_name).lower() # Remove special characters
    # We now try to scrape five different lyrics sites using predictable urls based on the artist and track names.
    artist_first_letter=artist_array[0][0]
    # The formatting for lyricsmode.com 
    artist_mode = re.sub(r'\s|\.|-', '_',artist_array[0]).replace(",","").lower()
    track_mode = re.sub(r'\s', '_',track_name)
    # The track formatting for songlyrics.com and metrolyrics.com is the same.
    # The artist formatting is different if the band starts with "The ".
    artist_song = re.sub(r'\s|\.', '-',artist_array[0]).replace(",","").lower()
    track_songmetro = re.sub(r'\s', '-',track_name)
    track_az = re.sub(r'\s', '',track_name)  # The track formatting for azlyrics.com
    artist_mmatch = re.sub(r'.And', '',artist_array[0]) # The artist formatting for musixmatch.com
    artist_mmatch = re.sub(r'\s|\.', '-',artist_mmatch).replace(",","").lower()
    # The urls for bands that start with "The " are formatted differently.
    if artist_array[0].startswith("The "):
        the_array = re.split(r'The ',artist_array[0])
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',the_array[1]).lower()
        artist_metro = re.sub(r'\.', '-',the_array[1]).replace(",","").replace(" ","-").lower()
    else:
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',artist_array[0]).lower()
        artist_metro=artist_song
    # Metro lyrics is inconsistent on whether it keeps "The " in the url so we have two possibilities.    
    url_metro_one = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_metro+'.html'
    url_metro_two = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_song+'.html'
    url_songlyrics = 'http://www.songlyrics.com/'+artist_song+'/'+track_songmetro+'-lyrics/'
    url_mode= 'http://www.lyricsmode.com/lyrics/'+artist_first_letter+'/'+artist_mode+'/'+track_mode+'.html'
    url_music_match = 'https://www.musixmatch.com/lyrics/' + artist_mmatch + '/' + track_songmetro
    url_azlyrics="http://www.azlyrics.com/lyrics/"+artist_az+"/"+track_az+".html"
    URLS=[url_metro_one,url_metro_two,url_songlyrics,url_mode, url_azlyrics, url_music_match]
    lyriclocations = ['lyrics-body-text', 'lyrics-body-text','songLyricsDiv','lyrics_text',"azlyrics",'mmatch']
    choice =""
    for u in range(0,len(URLS)):
        #We now cycle through each possbile url until we get a match.
        # We then scrape the lyric text from the website using beautiful soup.
        lyrics_request = urllib.request.Request(URLS[u],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0'})
        try:
            lyrics_web = urllib.request.urlopen(lyrics_request)
            lyrics_code = lyrics_web.read()
            soup = BeautifulSoup(lyrics_code, 'html.parser')
            if u==4:
                lyrics_array = soup.find_all("div", class_="")
            elif u == 5:
                lyrics_array = soup.find_all(class_="mxm-lyrics__content")
            else:
                lyrics_array = soup.find_all(id=lyriclocations[u])
            lyrics_text=""
            for lyrics in lyrics_array:
                lyrics_text += lyrics.get_text()
            # Some websites exist with no lyrics where the text is "\r\n ", "\r\r\n ", "\r\r\r\n ", or "\r\r\r\r\n " or "\n".
            if lyrics_text != "" and lyrics_text != "\r\n " and lyrics_text != "\n" and lyrics_text != "\r\r\n " and lyrics_text != "\r\r\r\n " and lyrics_text != "\r\r\r\r\n ":
                choice=u
                break
        except urllib.request.HTTPError as e:
            lyrics_text=""
            pass        
    add_frame = {"artist":artist_raw, "track": track_raw, "lyrics":lyrics_text, "source":choice}
    lyrics_data = lyrics_data.append(add_frame,ignore_index=True)
    # In order to not overwhelm the websites we have a delay of 45 seconds every 50 requests.
    if row%50==0 and row!=lower_limit and row!=upper_limit:
        sleep(45)
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
lyrics_data.to_csv(lyrics_table)

features retrieved in 0.00 seconds


In [30]:
# Cleaning and Wrangling step
# Begin to join all the csv files together, some may require adding in an additional empty row.
lyrics_all=pd.DataFrame()
lyrics_one = pd.read_csv('lyrics-0-100.csv',  encoding='latin1')
lyrics_two = pd.read_csv('lyrics-100-1000.csv',  encoding='latin1')
lyrics_all = pd.concat([lyrics_one,lyrics_two])
lyrics_new = pd.read_csv('lyrics-1000-1096.csv',  encoding='latin1')
lyrics_all = pd.concat([lyrics_all,lyrics_new])

In [ ]:
lyrics_new=pd.read_csv('lyrics-27587-27588.csv',encoding='latin1')
lyrics_all = pd.concat([lyrics_all,lyrics_new])

In [ ]:
lyrics_merge=lyrics_all[["artist","lyrics","source","track"]].reset_index()
lyrics_merge.to_csv('all_lyrics.csv')

In [8]:
billboard_lyrics = pd.read_csv("all_lyrics.csv",encoding='latin1')

In [3]:
lyrics_sub=billboard_lyrics[["artist","lyrics","source","track"]]

In [5]:
# We have Lyrics Data for 82.60% of the tracks in the Billboard Top 100.
len(lyrics_sub[lyrics_sub.lyrics.notnull()])/len(lyrics_sub)*100

82.6047556908801

In [6]:
lyrics_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27588 entries, 0 to 27587
Data columns (total 4 columns):
artist    27588 non-null object
lyrics    22789 non-null object
source    22789 non-null float64
track     27588 non-null object
dtypes: float64(1), object(3)
memory usage: 862.2+ KB


In [20]:
source_data=pd.DataFrame()
lyrics_count = len(lyrics_sub[lyrics_sub.source ==0])+len(lyrics_sub[lyrics_sub.source==1])
source_data = source_data.append({"source":"metrolyrics.com","lyrics count":lyrics_count},ignore_index=True)
lyrics_sources = ["songlyrics.com","lyricsmode.com","azlyrics.com","musixmatch.com"] 
for u in range(0,len(lyrics_sources)):
    lyrics_count = len(lyrics_sub[lyrics_sub.source ==u+2])
    source_data = source_data.append({"source":lyrics_sources[u],"lyrics count":lyrics_count},ignore_index=True)
source_data

,lyrics count,source
0,14527.0,metrolyrics.com
1,6594.0,songlyrics.com
2,1650.0,lyricsmode.com
3,6.0,azlyrics.com
4,12.0,musixmatch.com


In [5]:
new_big_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304643 entries, 0 to 304642
Data columns (total 24 columns):
Unnamed: 0          304643 non-null int64
chartDate           304643 non-null object
title               304643 non-null object
artist              304643 non-null object
peakPos             304643 non-null int64
lastPos             304643 non-null int64
weeks               304643 non-null int64
rank                304643 non-null int64
change              304643 non-null object
spotifyID           258820 non-null object
acousticness        258820 non-null float64
danceability        258808 non-null float64
duration_ms         258820 non-null float64
energy              258820 non-null float64
id                  258820 non-null object
instrumentalness    258820 non-null float64
key                 258820 non-null float64
liveness            258820 non-null float64
loudness            258820 non-null float64
mode                258820 non-null float64
speechiness         25880

In [6]:
audio_features=new_big_data[["title","artist","chartDate","peakPos","lastPos","weeks","rank","change","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness", "mode","speechiness","tempo","time_signature","valence"]]

In [7]:
audio_features.describe()

,peakPos,lastPos,weeks,rank,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,304643.000000,304643.000000,304643.000000,304643.000000,258820.000000,258808.000000,2.588200e+05,258820.000000,258820.000000,258820.000000,258820.000000,258820.000000,258820.000000,258808.000000,258820.000000,258781.000000,258808.000000
mean,30.760100,43.232797,13.902463,50.503392,0.270276,0.605626,2.286347e+05,0.632491,0.034109,5.224256,0.192357,-8.366804,0.729318,0.069378,120.303540,3.940718,0.604654
std,26.145309,30.145844,8.151371,28.866553,0.272556,0.151662,6.246360e+04,0.198102,0.139918,3.551212,0.164467,3.553880,0.444313,0.074781,28.090947,0.297135,0.236085
min,1.000000,0.000000,0.000000,1.000000,0.000001,0.078200,3.706700e+04,0.016700,0.000000,0.000000,0.013600,-29.110000,0.000000,0.021600,0.000000,1.000000,0.033500
25%,8.000000,16.000000,8.000000,26.000000,0.039800,0.505000,1.878670e+05,0.490000,0.000000,2.000000,0.089300,-10.623000,0.000000,0.032000,99.233000,4.000000,0.422000
50%,24.000000,42.000000,13.000000,51.000000,0.165000,0.616000,2.256670e+05,0.653000,0.000005,5.000000,0.130000,-7.822000,1.000000,0.040900,118.854000,4.000000,0.626000
75%,50.000000,69.000000,19.000000,76.000000,0.454000,0.712000,2.600670e+05,0.792000,0.000505,8.000000,0.249000,-5.636000,1.000000,0.065700,135.958000,4.000000,0.802000
max,100.000000,110.000000,87.000000,100.000000,0.995000,0.986000,1.367093e+06,0.998000,0.989000,11.000000,0.993000,2.291000,1.000000,0.898000,217.682000,5.000000,0.990000


In [94]:
#### Remember to remove "We do not have the lyrics yet" from lyrics column and starts with CHORUS

In [ ]:
# Analysis

In [ ]:
How does one go about creating a data story? You have some pointers from the material you've just gone through, but they’re probably a bit on the abstract side when you’re just getting started. Also, storytelling is an art, so you have to get your imagination going. Here are some pointers to get those creative juices flowing. In the following sections we will work step-by-step to create your first Data Story

        Get going by asking the following questions and looking for the answers with some code and plots:

            Can you count something interesting?

            Can you find some trends (high, low, increase, decrease, anomalies)?

            Can you make a bar plot or a histogram?

            Can you compare two related quantities?

            Can you make a scatterplot?

            Can you make a time-series plot?

        Having made these plots, what are some insights you get from them? Do you see any correlations? Is there a hypothesis you would like to investigate further? What other questions do they lead you to ask?

        By now you’ve asked a bunch of questions, and found some neat insights. Is there an interesting narrative, a way of presenting the insights using text and plots from the above, that tells a compelling story? As you work out this story, what are some other trends/relationships you think will make it more complete?

Submission Requirements: Submit links to a Github repository containing a Jupyter notebook. The notebook should contain:

    The questions you asked
    The trends you investigated
    The resulting visualizations and conclusions.

In case the data set is too large to commit to Github, please include a link to the data set inside the Jupyter notebook.

In [11]:
#https://dreference.blogspot.com/2016/09/positive-ve-words-and-adjectives-list.html
posi=pd.read_csv("posi.txt")
plist=list(posi.values.flatten())
plist = [x.lower() for x in plist]
plist.append('abiding')
negi=pd.read_csv("neg.txt")
nlist=list(negi.values.flatten())
nlist = [x.lower() for x in nlist]
nlist.append('abandoned')

In [9]:
#Find the common words
lyrics_list=billboard_lyrics[billboard_lyrics.lyrics.notnull()].reset_index()
lower_limit =0
upper_limit =len(lyrics_list)
common_words = pd.DataFrame()
rgx = re.compile("(\w[\w']*\w|\w)")
for row in range(lower_limit,upper_limit):
    song_lyrics=lyrics_list.get_value(row,"lyrics")
    words = rgx.findall(song_lyrics)
    word_list = [word.lower() for word in words] 
    word_counts = Counter(word_list)
    add_frame = {"artist":lyrics_list.get_value(row,"artist"), "track":lyrics_list.get_value(row,"track"), "lyrics":song_lyrics,"common words":word_counts.most_common()}
    common_words = common_words.append(add_frame,ignore_index=True)

In [12]:
#Create Sentiment lyrical feature
emo_data=pd.DataFrame()
for row in range(0,len(common_words)):
    the_word_count=common_words.get_value(row,"common words")
    positivity=0
    total_words=0
    for word in the_word_count:
        total_words+=word[1]
        if word[0] in plist:
            positivity += word[1]
        elif word[0] in nlist:
             positivity -=word[1]
    add_frame = {"artist":common_words.get_value(row,"artist"), "track":common_words.get_value(row,"track"), "lyrics":common_words.get_value(row,"lyrics"),"positivity":positivity/total_words, 'common_words':common_words.get_value(row,"common words")}
    emo_data = emo_data.append(add_frame,ignore_index=True)

In [13]:
emo_data.to_csv("sentiment_data.csv")

In [2]:
emo_data = pd.read_csv("sentiment_data.csv", encoding='latin1')

In [15]:
emo_data.sort_values(by=["positivity","artist","track"],ascending=True).head(10)

,artist,common_words,lyrics,positivity,track
834,Jon Bellion,"[(low, 84), (lie, 17), (i, 11), (that, 11), (t...","\r\r\r\r\nI, was the knight in shining armor i...",-0.292683,All Time Low
3546,Kiss,"[(crazy, 48), (are, 16), (these, 7), (nights, ...","\r\r\r\r\nPeople try to take my soul away, but...",-0.200935,Crazy Crazy Nights
10960,Dazz Band,"[(down, 11), (let, 9), (it, 9), (all, 9), (blo...",\r\r\r\r\nLet it all blow. Heave-ho heave-ho l...,-0.196429,Let It All Blow
19859,Color Me Badd,"[(so, 49), (bad, 48), (time, 39), (chance, 38)...",(Time)\r\r\r\r\n(Chance)\r\r\r\r\n(Time)\r\r\r...,-0.192683,Time And Chance
5649,Dee Dee Warwick,"[(crazy, 22), (foolish, 20), (she, 10), (my, 1...",Foolish.Foolish... Foolish... Foolish... Foooo...,-0.188889,Foolish Fool
15811,Barbara Mason,"[(sad, 32), (you, 19), (girl, 15), (over, 13),...","Sad, sad, girl (sad girl)\r\r\r\r\n\r\r\r\r\nI...",-0.183432,"Sad, Sad Girl"
4802,Sean Kingston,"[(dumb, 90), (you, 23), (b, 20), (wh, 15), (i,...","\r\r\r\r\nGirl you make me dumb, dumb, dumb, d...",-0.177500,Dumb Love
1704,Sean Kingston Featuring Chris Brown & Wiz Khalifa,"[(beat, 125), (it, 43), (you, 22), (me, 20), (...","\r\r\r\r\nHe ain't fly though, he don't even d...",-0.174520,Beat It
14518,The Doors,"[(you're, 18), (when, 16), (strange, 11), (are...",\r\r\r\r\nPeople are strange when you're a str...,-0.168317,People Are Strange
13889,Jack Scott,"[(little, 22), (one, 22), (oh, 18), (i, 5), (s...",Oh little one (oh little one)\r\r\r\r\nOh litt...,-0.166667,"Oh, Little One"


In [103]:
sorted_dat =emo_data.sort_values(by=["positivity","track", "artist"],ascending=False)

In [112]:
lyric_count=pd.merge(billboard_lyrics,common_words,how='left', on=['artist', 'track',"lyrics"])

In [ ]:
sad = pd.read_excel("sad_words.xls")
slist=list(sad.dropna().values.flatten())
slist = [x for x in slist if x != '\xa0']
slist = [x.lower() for x in slist]
slist =  [x.strip() for x in slist]

In [ ]:
happy=pd.read_excel("happy_words.xls")
hlist=list(happy.dropna().values.flatten())
hlist = [x for x in hlist if x != '\xa0']
hlist = [x.lower() for x in hlist]
hlist =  [x.strip() for x in hlist]

In [ ]:
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
hlist=remove_duplicates(hlist)
slist=remove_duplicates(slist)